# Quick Training Demo - HAR Theft Detection (Google Colab)

This notebook demonstrates end-to-end training on a small dataset.
Can be run in Google Colab with GPU for fast training.

## Setup

In [ ]:
# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except:
    IN_COLAB = False
    print("Running locally")

# Clone repository if in Colab
if IN_COLAB:
    !git clone https://github.com/YOUR_USERNAME/har_theft_detection.git
    %cd har_theft_detection

# Install dependencies
!pip install -q torch torchvision pytorchvideo timm opencv-python pandas numpy scikit-learn matplotlib seaborn tensorboard albumentations tqdm einops

## Check GPU

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## Create Dummy Dataset

In [ ]:
from src.datasets import create_dummy_dataset

# Create a small dataset for quick training
train_csv, val_csv = create_dummy_dataset(
    output_dir='data',
    num_videos_per_class=10,  # 10 videos per class
    num_frames=32
)

print(f"\nTrain CSV: {train_csv}")
print(f"Val CSV: {val_csv}")

## Train Model (Quick Demo)

In [ ]:
# Train a simple 3D CNN for quick demo (faster than transformer)
!python src/train.py \
    --train_csv data/metadata_train.csv \
    --val_csv data/metadata_val.csv \
    --clips_dir data/clips \
    --model_name simple3d \
    --epochs 10 \
    --batch_size 4 \
    --lr 3e-4 \
    --num_frames 32 \
    --output_dir results/checkpoints/demo \
    --log_dir results/logs/demo

## View Training Results

In [ ]:
import json
from IPython.display import Image, display

# Load metrics
with open('results/metrics/simple3d_metrics.json', 'r') as f:
    metrics = json.load(f)

print("Final Metrics:")
print(f"  Accuracy: {metrics['accuracy']:.4f}")
print(f"  F1 Score: {metrics['f1_macro']:.4f}")
print(f"  Precision: {metrics['precision_macro']:.4f}")
print(f"  Recall: {metrics['recall_macro']:.4f}")

# Display plots
print("\nConfusion Matrix:")
display(Image('results/figures/simple3d_confusion_matrix.png'))

print("\nTraining Curves:")
display(Image('results/figures/simple3d_training_curves.png'))

## Test Inference on Sample Video

In [ ]:
# Create a test video from dummy data
import cv2
import numpy as np

# Generate a simple test video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('test_video.mp4', fourcc, 10, (224, 224))

for i in range(100):
    frame = np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8)
    out.write(frame)

out.release()
print("Test video created: test_video.mp4")

In [ ]:
# Run inference
!python src/infer_video.py \
    --video test_video.mp4 \
    --checkpoint results/checkpoints/demo/best_acc.pth \
    --model_name simple3d \
    --output_video output_video.mp4 \
    --output_csv predictions.csv

In [ ]:
# View predictions
import pandas as pd

df = pd.read_csv('predictions.csv')
print("Predictions:")
print(df.head(10))

print("\nPrediction distribution:")
print(df['predicted_label'].value_counts())

## Train High-Accuracy Model (Optional - Takes Longer)

In [ ]:
# Train TimeSformer for better accuracy (requires more time and memory)
# Uncomment to run:

# !python src/train.py \
#     --train_csv data/metadata_train.csv \
#     --val_csv data/metadata_val.csv \
#     --clips_dir data/clips \
#     --model_name timesformer \
#     --epochs 25 \
#     --batch_size 2 \
#     --lr 1e-4 \
#     --num_frames 32 \
#     --output_dir results/checkpoints/timesformer \
#     --log_dir results/logs/timesformer

## Download Results

In [ ]:
if IN_COLAB:
    # Zip results for download
    !zip -r results.zip results/
    
    from google.colab import files
    files.download('results.zip')
    print("Results downloaded!")